<a href="https://colab.research.google.com/github/franklinokech/Geographic_distance_calculator/blob/master/Find_Distance_Between_Farms_and_CCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# import key libraries
import pandas as pd
import numpy as np
import sklearn.neighbors

In [28]:
# Import Libraries for connecting to Google drive
!pip install  -q gspread

In [29]:
# Authenticate to Google drive and get the required dataset
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [30]:
# Import SSOT
# Get the google sheet file for jaribuni
sh = gc.open('SSOT Shamba Person_MASTER')
# Select Spreadsheet
# By title
worksheet = sh.worksheet('Shamba')
data = worksheet.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)

# Preview dataframe

In [31]:
df = df[['kcode', 'latitude', 'longitude']]



In [32]:
# convert to correct datatypes
df.latitude = pd.to_numeric(df.latitude)
df.longitude = pd.to_numeric(df.longitude)



In [33]:
# Import SSOT
# Get the google sheet file for jaribuni
sh = gc.open('komaza gps cordinates')
# Select Spreadsheet
# By title
worksheet = sh.worksheet('data')
data = worksheet.get_all_values()
headers = data.pop(0)

df_2 = pd.DataFrame(data, columns=headers)

# Preview dataframe


In [34]:
df_2.drop(df_2.index[0], inplace=True)

In [35]:
df_2.drop(['point', 'routeXL'], axis=1, inplace=True)

In [36]:
cols_to_rename = {
    'lat': 'latitude',
    'lon': 'longitude'
}

df_2.rename(columns=cols_to_rename, inplace=True)

In [37]:
df_2.latitude = pd.to_numeric(df_2.latitude)
df_2.longitude = pd.to_numeric(df_2.longitude)

In [38]:
# add columns with radians for latitude and longitude
df[['latitude_rad','longitude_rad']] = (
    np.radians(df.loc[:,['latitude','longitude']])
)
df_2[['latitude_rad','longitude_rad']] = (
    np.radians(df_2.loc[:,['latitude','longitude']])
)

In [39]:
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')
dist_matrix = (dist.pairwise
    (df[['latitude_rad','longitude_rad']],
     df_2[['latitude_rad','longitude_rad']])*6371
)
# Note that 6371 is the radius of the earth in km
df_dist_matrix = (
    pd.DataFrame(dist_matrix,index=df['kcode'], 
                 columns=df_2['name'])
)

In [41]:
# Unpivot this dataframe from wide format to long format.
# When you unpivot, the data in the pivot table becomes a
# column named 'value'. Rename this column to 'miles' for clarity.
df_dist_long = (
    pd.melt(df_dist_matrix.reset_index(),id_vars='kcode')
)
df_dist_long = df_dist_long.rename(columns={'value':'km'})

In [43]:
from google.colab import auth
from gspread_dataframe import get_as_dataframe, set_with_dataframe
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())



# import gsheet with the data
# Spreadsheet ID:1K5p6xrOy8t-v0vfYWDXm9WqIIgdaPtMNLoNjbXFhHk0

sh = gc.open('ROUTING DISTANCE ESTIMATE')

# Select Spreadsheet
# By title
worksheet = sh.worksheet('kcode_cca_distance')

In [44]:
# Append Dataframe to Sheet
set_with_dataframe(worksheet, df_dist_long)